In [2]:
import requests

url = "https://raw.githubusercontent.com/karpathy/ng-video-lecture/master/input.txt"

# Fazendo a requisição GET para obter o conteúdo do arquivo
response = requests.get(url)

# Verificando se a requisição foi bem-sucedida (status code 200)
if response.status_code == 200:
    # Salvar o conteúdo do arquivo
    with open("input.txt", "wb") as file:
        file.write(response.content)
    print("Arquivo baixado com sucesso!")
else:
    print("Erro ao baixar o arquivo. Status code:", response.status_code)


Arquivo baixado com sucesso!


In [3]:
# Explorando os dados

with open('input.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [4]:
print("Tamanho do datatset(caracteres): ", len(text))

Tamanho do datatset(caracteres):  1115394


In [6]:
#Printa as primeiros 1000 caracteres
print(text[:1000])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.
Is't a verdict?

All:
No more talking on't; let it be done: away, away!

Second Citizen:
One word, good citizens.

First Citizen:
We are accounted poor citizens, the patricians good.
What authority surfeits on would relieve us: if they
would yield us but the superfluity, while it were
wholesome, we might guess they relieved us humanely;
but they think we are too dear: the leanness that
afflicts us, the object of our misery, is as an
inventory to particularise their abundance; our
sufferance is a gain to them Let us revenge this with
our pikes, ere we become rakes: for the gods know I
speak this in hunger for bread, not in thirst for revenge.



In [9]:
#Pegar o Vocabulario do Texto
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))


 !$&',-.3:;?ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz


In [8]:
print(vocab_size)

65


In [11]:
stoi = {ch:i for i,ch in enumerate(chars)}
itos = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # transforma um string em um  array de inteiros
decode = lambda l: ''.join(itos[i] for i in l) #transforma uma lista de inteiro em uma string
print(encode('Ola Pessoal'))
print(decode(encode('Ola Pessoal')))

[27, 50, 39, 1, 28, 43, 57, 57, 53, 39, 50]
Ola Pessoal
